# Optimization Model to Markdown Strategy

## Import data and pre-processing

In [2]:
import pandas as pd
import numpy as np

# Load the Excel file
data = pd.read_excel('../game_scraping/scraping_results.xlsx')

data.head(20)

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
1,1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
2,2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892
3,3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
4,4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892
5,5,6,48,239,1248,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
6,6,7,48,137,1111,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
7,7,8,36,261,850,"Your revenue: $85,752, Perfect foresight strat...",3,15.8,85752,101892
8,8,9,36,292,558,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892
9,9,10,36,294,264,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892


In [3]:
# Assuming each item's data is in blocks of 15 rows. Extract data for each item.
item_blocks = [data.iloc[i*15:(i+1)*15] for i in range(len(data) // 15)]
demand_lifts = []

In [4]:
# print sample item_block
item_blocks[1]

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
15,15,1,60,79,1921,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
16,16,2,60,59,1862,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
17,17,3,54,86,1776,"Your revenue: $79,188, Perfect foresight strat...",1,9.0,79188,87054
18,18,4,54,80,1696,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
19,19,5,48,74,1622,"Your revenue: $79,188, Perfect foresight strat...",2,9.0,79188,87054
20,20,6,36,219,1403,"Your revenue: $79,188, Perfect foresight strat...",3,9.0,79188,87054
21,21,7,36,120,1283,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
22,22,8,36,224,1059,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
23,23,9,36,130,929,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054
24,24,10,36,165,764,"Your revenue: $79,188, Perfect foresight strat...",0,9.0,79188,87054


## Calculate Demand Lift

In our dataset, there are 100 different items. Each item has a different demand at different price point. Given that the relative demand lift is the same, we extracted the demand lift for each price point for each item relative to the demand at week 1. Thereafter, we take the average of the demand lift for each price point to get 4 demand lift values.

In [5]:
# Calculate demand lift for each block (representing an item)
for block in item_blocks:
    week_1_sales = block[block['Week'] == 1]['Sales'].values[0]
    demand_lift = block.assign(Demand_Lift=lambda x: x['Sales'] / week_1_sales)
    demand_lifts.append(demand_lift)

# Combine all item blocks back into a single dataframe
demand_lifts_df = pd.concat(demand_lifts)

# Calculate the average demand lift for each price point across all items
average_demand_lifts = demand_lifts_df.groupby('Price')['Demand_Lift'].mean().reset_index()

# Output the results
print(average_demand_lifts)

   Price  Demand_Lift
0     36     1.772344
1     48     1.752019
2     54     1.316609
3     60     0.992055


In [6]:
demand_lifts_df.head(15)

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Demand_Lift
0,0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1.000000
1,1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,0.850575
2,2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892,1.689655
3,3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,0.517241
4,4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892,1.839080
5,5,6,48,239,1248,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,2.747126
6,6,7,48,137,1111,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1.574713
7,7,8,36,261,850,"Your revenue: $85,752, Perfect foresight strat...",3,15.8,85752,101892,3.000000
8,8,9,36,292,558,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,3.356322
9,9,10,36,294,264,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,3.379310


## Optimization Problem Formulation

Maximize the revenue:

$$ \\$60 \cdot 0.992055 \cdot d_{1} \cdot x_{60} + \\$54 \cdot 1.316609 \cdot d_{1} \cdot x_{54} + \\$48 \cdot 1.752019 \cdot d_{1} \cdot x_{48} + \\$36 \cdot 1.772344 \cdot d_{1} \cdot x_{36} $$

where:
- $d_{1}$ = initial demand at week 1
- $x_{60}, x_{54}, x_{48}, x_{36}$ = number of weeks the item is sold at \\$60, \\$54, \\$48, and \\$36 respectively

Subject to:
- $ 0.992055 \cdot d_{1} \cdot x_{60} + 1.316609 \cdot d_{1} \cdot x_{54} + 1.752019 \cdot d_{1} \cdot x_{48} + 1.772344 \cdot d_{1} \cdot x_{36}$
- $x_{60} + x_{54} + x_{48} + x_{36} \leq 15$
- $x_{60} \geq 1$
- $x_{54}, x_{48}, x_{36} \geq 0$

## Running the optimization problem using a series of initial demand

Now, we will optimise using the above formulation to determine a suitable markdown strategy for each initial demand, $d_1$, i.e. the demand for a particular item at week 1

In [7]:
# Extract week 1 sales (assuming data is loaded into a DataFrame named 'df')
week_1_sales = data[data['Week'] == 1]['Sales'].values

# Get unique sales values using numpy
unique_sales = np.unique(week_1_sales)

# Print or further use the unique sales values
print(unique_sales)

print(len(unique_sales))

[ 17  25  30  36  44  47  48  49  51  53  54  55  56  61  65  66  67  68
  69  70  71  72  74  76  79  81  83  84  85  86  87  89  90  92  94  96
  98  99 100 101 102 103 104 105 107 108 110 111 112 114 116 117 119 122
 125 127 128 129 133 139 167]
61


Given that there are 61 unique $d_1$ values within the list of 100 items, we will build a series of optimisation model that runs from $d_1$ = 15, 25, 35 ... 175 to recommend the appropriate strategy depending on which buckets the initial $d_1$ falls into.

In [10]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Create buckets for week_1_sales
week_1_sales = np.arange(15, 185, 10)

# Placeholder for results
optimization_results = []

for d1 in week_1_sales:
    # Create a new model
    model = gp.Model('PricingStrategy')
    model.setParam('OutputFlag', 0)  # Suppress the optimization log

    # Decision variables
    x60 = model.addVar(lb=1, vtype=GRB.INTEGER, name="x60")
    x54 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x54")
    x48 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x48")
    x36 = model.addVar(lb=0, vtype=GRB.INTEGER, name="x36")

    # Objective function
    model.setObjective(60 * 0.992055 * d1 * x60 +
                       54 * 1.316609 * d1 * x54 +
                       48 * 1.752019 * d1 * x48 +
                       36 * 1.772344 * d1 * x36, GRB.MAXIMIZE)

    # Constraints
    model.addConstr(0.992055 * d1 * x60 + 1.316609 * d1 * x54 + 1.752019 * d1 * x48 + 1.772344 * d1 * x36 <= 2000, "c0")
    model.addConstr(x60 + x54 + x48 + x36 <= 15, "c1")

    # Solve model
    model.optimize()

    # Store results if an optimal solution is found
    if model.status == GRB.OPTIMAL:
        results = {
            'd1': d1,
            'x60': x60.X,
            'x54': x54.X,
            'x48': x48.X,
            'x36': x36.X,
            'Objective': model.ObjVal  # Capture the objective function value
        }
        optimization_results.append(results)

# Convert results into a DataFrame
results_df = pd.DataFrame(optimization_results)

# Output results
print(results_df)

Restricted license - for non-production use only - expires 2025-11-24
     d1   x60  x54   x48  x36     Objective
0    15   1.0 -0.0  14.0 -0.0   18553.20102
1    25   1.0 -0.0  14.0 -0.0   30922.00170
2    35   1.0 -0.0  14.0 -0.0   43290.80238
3    45   1.0 -0.0  14.0 -0.0   55659.60306
4    55   1.0 -0.0  14.0 -0.0   68028.40374
5    65   1.0 -0.0  14.0 -0.0   80397.20442
6    75   1.0 -0.0  14.0 -0.0   92766.00510
7    85   2.0  3.0  10.0 -0.0   99731.04213
8    95   3.0  7.0   5.0 -0.0  104189.60289
9   105   5.0  8.0   2.0 -0.0  108631.46826
10  115  10.0  3.0   2.0 -0.0  112322.51043
11  125  13.0  1.0   1.0 -0.0  116124.58725
12  135  13.0  0.0   1.0 -0.0  115816.47462
13  145  12.0  0.0   1.0 -0.0  115764.59424
14  155  13.0 -0.0  -0.0 -0.0  119939.44950
15  165  12.0 -0.0  -0.0 -0.0  117856.13400
16  175  10.0  1.0   0.0 -0.0  116607.73005


In [11]:
print(results_df.iloc[:,0:5])

     d1   x60  x54   x48  x36
0    15   1.0 -0.0  14.0 -0.0
1    25   1.0 -0.0  14.0 -0.0
2    35   1.0 -0.0  14.0 -0.0
3    45   1.0 -0.0  14.0 -0.0
4    55   1.0 -0.0  14.0 -0.0
5    65   1.0 -0.0  14.0 -0.0
6    75   1.0 -0.0  14.0 -0.0
7    85   2.0  3.0  10.0 -0.0
8    95   3.0  7.0   5.0 -0.0
9   105   5.0  8.0   2.0 -0.0
10  115  10.0  3.0   2.0 -0.0
11  125  13.0  1.0   1.0 -0.0
12  135  13.0  0.0   1.0 -0.0
13  145  12.0  0.0   1.0 -0.0
14  155  13.0 -0.0  -0.0 -0.0
15  165  12.0 -0.0  -0.0 -0.0
16  175  10.0  1.0   0.0 -0.0


## Testing the Strategy

In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Print the versions of selenium and webdriver-manager
import selenium
import webdriver_manager

print("Selenium version:", selenium.__version__)
print("WebDriver Manager version:", webdriver_manager.__version__)

# Initialize the driver with correct setup
def init_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Opens the browser window maximized
    chrome_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

try:
    driver = init_driver()
    print("Driver initialized successfully.")
    driver_version = driver.capabilities['chrome']['chromedriverVersion'].split(' ')[0]
    browser_version = driver.capabilities['browserVersion']
    print("ChromeDriver version:", driver_version)
    print("Browser version:", browser_version)
    driver.quit()
except Exception as e:
    print("Failed to initialize driver:", str(e))


Selenium version: 4.21.0
WebDriver Manager version: 4.0.1
Driver initialized successfully.
ChromeDriver version: 125.0.6422.60
Browser version: 125.0.6422.61


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import time

def init_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Opens the browser window maximized
    chrome_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

# Define the strategy DataFrame with d1 and actions for each week
data = {
    'd1': np.arange(15, 180, 10),
    'Actions': []
}

# Pricing strategy details
x60_weeks = [1] * 7 + [2, 3, 5, 10, 13, 13, 12, 13, 12, 10]
x54_weeks = [0] * 7 + [3, 7, 8, 3, 1, 0, 0, 0, 0, 1]
x48_weeks = [14] * 7 + [10, 5, 2, 2, 1, 1, 1, 0, 0, 0]
x36_weeks = [0] * 17

for i in range(len(data['d1'])):
    actions = []
    if x60_weeks[i] > 0:
        actions += ['maintain'] * x60_weeks[i]
    if x54_weeks[i] > 0:
        actions += ['ten'] + ['maintain'] * (x54_weeks[i] - 1)
    if x48_weeks[i] > 0:
        actions += ['twenty'] + ['maintain'] * (x48_weeks[i] - 1)
    if x36_weeks[i] > 0:
        actions += ['forty'] + ['maintain'] * (x36_weeks[i] - 1)
    data['Actions'].append(actions)

strategy_df = pd.DataFrame(data)

# Initialize the driver and navigate to the game
driver = init_driver()

# Prepare to collect all run results
all_runs = []

for _ in range(10):
    driver.get("http://www.randhawa.us/games/retailer/nyu.html")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table")))
    try:
        first_sales_value_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//table[@id='result-table']/thead/tr[2]/td[3]"))
        )
        first_sales_value = int(first_sales_value_element.text)
        closest_strategy = strategy_df.iloc[(strategy_df['d1'] - first_sales_value).abs().argsort()[:1]]
        actions = closest_strategy['Actions'].values[0]

        buttons = {
            "maintain": driver.find_element(By.ID, "maintainButton"),
            "ten": driver.find_element(By.ID, "tenButton"),
            "twenty": driver.find_element(By.ID, "twentyButton"),
            "forty": driver.find_element(By.ID, "fortyButton")
        }

        for action in actions:
            buttons[action].click()
            time.sleep(1)

        all_runs.append({'Run': _ + 1, 'First Sales Value': first_sales_value, 'Strategy': closest_strategy.iloc[0].to_dict()})

    except Exception as e:
        print(f"Error on run {_ + 1}: {str(e)}")

driver.quit()

runs_df = pd.DataFrame(all_runs)
print(runs_df)

   Run  First Sales Value                                           Strategy
0    1                 67  {'d1': 65, 'Actions': ['maintain', 'twenty', '...
1    2                 89  {'d1': 85, 'Actions': ['maintain', 'maintain',...
2    3                 71  {'d1': 75, 'Actions': ['maintain', 'twenty', '...
3    4                 93  {'d1': 95, 'Actions': ['maintain', 'maintain',...
4    5                154  {'d1': 155, 'Actions': ['maintain', 'maintain'...
5    6                127  {'d1': 125, 'Actions': ['maintain', 'maintain'...
6    7                 72  {'d1': 75, 'Actions': ['maintain', 'twenty', '...
7    8                 75  {'d1': 75, 'Actions': ['maintain', 'twenty', '...
8    9                 95  {'d1': 95, 'Actions': ['maintain', 'maintain',...
9   10                 89  {'d1': 85, 'Actions': ['maintain', 'maintain',...


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import time

def init_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Opens the browser window maximized
    chrome_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

# Define the strategy DataFrame with d1 and actions for each week
data = {
    'd1': np.arange(15, 180, 10),
    'Actions': []
}

# Pricing strategy details
x60_weeks = [1] * 7 + [2, 3, 5, 10, 13, 13, 12, 13, 12, 10]
x54_weeks = [0] * 7 + [3, 7, 8, 3, 1, 0, 0, 0, 0, 1]
x48_weeks = [14] * 7 + [10, 5, 2, 2, 1, 1, 1, 0, 0, 0]
x36_weeks = [0] * 17

for i in range(len(data['d1'])):
    actions = []
    if x60_weeks[i] > 0:
        actions += ['maintain'] * (x60_weeks[i] - 1)
    if x54_weeks[i] > 0:
        actions += ['ten'] + ['maintain'] * (x54_weeks[i] - 1)
    if x48_weeks[i] > 0:
        actions += ['twenty'] + ['maintain'] * (x48_weeks[i] - 1)
    if x36_weeks[i] > 0:
        actions += ['forty'] + ['maintain'] * (x36_weeks[i] - 1)
    data['Actions'].append(actions)

strategy_df = pd.DataFrame(data)

# Initialize the driver and navigate to the game
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table")))

# Collect all run results
all_runs = []
attempts = 3

for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "practiceButton"))).click()
    time.sleep(1)  # Ensure the game has reset

    chosenchoices = []
    run_data = []

    # Fetch initial demand and find closest strategy
    first_sales_value = int(WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//table[@id='result-table']/thead/tr[2]/td[3]"))
    ).text)
    closest_strategy = strategy_df.iloc[(strategy_df['d1'] - first_sales_value).abs().argsort()[:1]]
    actions = closest_strategy['Actions'].values[0]

    buttons = {
        "maintain": driver.find_element(By.ID, "maintainButton"),
        "ten": driver.find_element(By.ID, "tenButton"),
        "twenty": driver.find_element(By.ID, "twentyButton"),
        "forty": driver.find_element(By.ID, "fortyButton")
    }
    
    # Execute strategy and capture data
    for i, action in enumerate(actions):
        buttons[action].click()
        time.sleep(1)  # Allow time for the table to update

    
    text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table"))).text
            
    lines = text.split('\n')
    # Slice the lines from the specified start line onwards
    data_lines = lines[1:]  # adjust the index based on where you want to start

    # Split each line into components based on spaces, assuming data is space-separated
    data = [line.split() for line in data_lines]

    df = pd.DataFrame(data, columns=['Week', 'Price', 'Sales', 'Remaining Inventory'])
    
    all_data = pd.concat(df, ignore_index=True)

driver.quit()

print(f"Total rows captured over all attempts: {len(all_data)}")
print(all_data)


Attempt 1/3


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [17]:
def to_excel(df, name):
    with pd.ExcelWriter(name + '.xlsx') as writer:
        df.to_excel(writer, 'Sheet1')
driver = init_driver()


data = {
    'd1': np.arange(15, 180, 10),
    'Actions': []
}


x60_weeks = [1] * 7 + [2, 3, 5, 10, 13, 13, 12, 13, 12, 10]
x54_weeks = [0] * 7 + [3, 7, 8, 3, 1, 0, 0, 0, 0, 1]
x48_weeks = [14] * 7 + [10, 5, 2, 2, 1, 1, 1, 0, 0, 0]
x36_weeks = [0] * 17

for i in range(len(data['d1'])):
    actions = []
    if x60_weeks[i] > 0:
        actions += ['maintain'] * x60_weeks[i]
    if x54_weeks[i] > 0:
        actions += ['ten'] + ['maintain'] * (x54_weeks[i] - 1)
    if x48_weeks[i] > 0:
        actions += ['twenty'] + ['maintain'] * (x48_weeks[i] - 1)
    if x36_weeks[i] > 0:
        actions += ['forty'] + ['maintain'] * (x36_weeks[i] - 1)
    data['Actions'].append(actions)

strategy_df = pd.DataFrame(data)


all_runs = []
all_data = []


def execute_strategy(driver, actions):
    buttons = {
        "maintain": driver.find_element(By.ID, "maintainButton"),
        "ten": driver.find_element(By.ID, "tenButton"),
        "twenty": driver.find_element(By.ID, "twentyButton"),
        "forty": driver.find_element(By.ID, "fortyButton")
    }
    for action in actions:
        buttons[action].click()
        time.sleep(1)  


def scrape_data(driver):
    data = driver.find_element(By.ID, "result-table").text
    splitdata = data.split("\n")
    columns = splitdata[0].split(" ")
    values_split = [row.split(" ") for row in splitdata[1:]]

    table = pd.DataFrame(values_split, columns=columns[:4])
    return table


attempts = 10
for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    driver.get("http://www.randhawa.us/games/retailer/nyu.html")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "practiceButton"))).click()
    time.sleep(1) 
    

    first_sales_value = int(WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//table[@id='result-table']/thead/tr[2]/td[3]"))
    ).text)
    closest_strategy = strategy_df.iloc[(strategy_df['d1'] - first_sales_value).abs().argsort()[:1]]
    actions = closest_strategy['Actions'].values[0]
    

    execute_strategy(driver, actions)
    
    table = scrape_data(driver)
    
    all_data.append(table)

driver.quit()

final_table = pd.concat(all_data, ignore_index=True)

to_excel(final_table, "scraping_results_second_ed")

# Print the final table
print(final_table)


Attempt 1/10
Attempt 2/10
Attempt 3/10
Attempt 4/10
Attempt 5/10
Attempt 6/10
Attempt 7/10
Attempt 8/10
Attempt 9/10
Attempt 10/10
    Week Price Sales Remaining
0      1    60    84      1916
1      2    60    80      1836
2      3    60    91      1745
3      4    54    92      1653
4      5    54    90      1563
..   ...   ...   ...       ...
145   11    54   119       745
146   12    48   152       593
147   13    48   125       468
148   14    48   139       329
149   15    48   114       215

[150 rows x 4 columns]


/var/folders/zh/7hbjyl3x1y953yvj5t_7dbbw0000gn/T/ipykernel_84381/4241300513.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Sheet1')


In [18]:
# Initialize the driver and navigate to the game
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")
text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "result-table"))).text
            
lines = text.split('\n')
# Slice the lines from the specified start line onwards
data_lines = lines[1:]  # adjust the index based on where you want to start

# Split each line into components based on spaces, assuming data is space-separated
data = [line.split() for line in data_lines]

df

,Week,Price,Sales,Remaining Inventory
0,1,60,97,1903
1,2,60,78,1825
2,3,60,128,1697
3,4,54,82,1615
4,5,54,115,1500
5,6,54,114,1386
6,7,54,66,1320
7,8,54,164,1156
8,9,54,155,1001
9,10,54,87,914


In [19]:
data

[['1', '60', '132', '1868']]

In [23]:
driver = init_driver()
attempts = 10
for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    driver.get("http://www.randhawa.us/games/retailer/nyu.html")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "practiceButton"))).click()
    time.sleep(1) 
    

    first_sales_value = int(WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//table[@id='result-table']/thead/tr[2]/td[3]"))
    ).text)
    closest_strategy = strategy_df.iloc[(strategy_df['d1'] - first_sales_value).abs().argsort()[:1]]
    actions = closest_strategy['Actions'].values[0]
    

    execute_strategy(driver, actions)
    

    table = scrape_data(driver)
    

    table['Revenue'] = table['Price'].astype(float) * table['Sales'].astype(float)
    revenue = table['Revenue'].sum()
    perfect_revenue = 2000 * (int(table['Week'].max()) / 15)
    table['% diff'] = ((revenue - perfect_revenue) / perfect_revenue) * 100
    

    all_data.append(table)

driver.quit()


final_table = pd.concat(all_data, ignore_index=True)


mean_diff = final_table['% diff'].mean()
std_diff = final_table['% diff'].std()


to_excel(final_table, "scraping_results_third_ed")


summary_df = pd.DataFrame({'mean_diff': [mean_diff], 'std_diff': [std_diff]})
to_excel(summary_df, "summary_statistics")

# Print the results
print(f"Mean % diff: {mean_diff}")
print(f"Standard Deviation of % diff: {std_diff}")

Attempt 1/10
Attempt 2/10
Attempt 3/10
Attempt 4/10
Attempt 5/10
Attempt 6/10
Attempt 7/10
Attempt 8/10
Attempt 9/10
Attempt 10/10
Mean % diff: 8451.124161073825
Standard Deviation of % diff: 886.310243456286


/var/folders/zh/7hbjyl3x1y953yvj5t_7dbbw0000gn/T/ipykernel_84381/4241300513.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Sheet1')
